In [8]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# def scrape_ucl_season(url):
#     html=requests.get(url).text
#     soup=BeautifulSoup(html,"html.parser")

#     matches=[]

#     table=soup.find("table",{"id":"sched_8_2023-2024_8"}) or \
#           soup.find("table",{"id":"sched_8_2023-2024_8"}) or \
#           soup.find('table')

#     for row in table.find_all('tr'):
#         cols=row.find_all('td')


#         if len(cols)<5:
#             continue


#         date=cols[0].text.strip()
#         home=cols[2].text.strip()
#         score=cols[3].text.strip()
#         away=cols[4].text.strip()
        

#         if score =="":
#             continue

#         matches.append({
#             "date":date,
#             "home_team":home,
#             "away_team":away,
#             "score":score
#         })

#         return pd.Dataframe(matches)
    
# ucl_2023= "https://fbref.com/en/comps/8/2023-2024/schedule/2023-2024-Champions-League-Scores-and-Fixtures"
# # ucl_2024 = "https://fbref.com/en/comps/8/2024-2025/schedule/2024-2025-Champions-League-Scores-and-Fixtures"  
# ucl_2024="https://fbref.com/en/comps/8/2023-2024/schedule/2023-2024-Champions-League-Scores-and-Fixtures",
# df_2023=scrape_ucl_season(ucl_2024)

# df_2023.head()

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

HEADERS = {"User-Agent": "Mozilla/5.0"}

def scrape_ucl(url):
    # download page
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # Find the first table on the page (FBref schedules always use this)
    table = soup.find("table")
    if table is None:
        raise Exception("No table found on the page.")

    matches = []

    for row in table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) < 5:
            continue

        date = cols[0].get_text(strip=True)
        home = cols[2].get_text(strip=True)
        score = cols[3].get_text(strip=True)
        away = cols[4].get_text(strip=True)

        # Skip future fixtures (score missing)
        if not re.search(r"\d+\s*[–-]\s*\d+", score):
            continue

        # Extract goals
        m = re.search(r"(\d+)\s*[–-]\s*(\d+)", score)
        home_goals = int(m.group(1))
        away_goals = int(m.group(2))

        matches.append({
            "date": date,
            "home_team": home,
            "away_team": away,
            "home_goals": home_goals,
            "away_goals": away_goals
        })

    return pd.DataFrame(matches)


# REAL WORKING FBREF URLS (NO TUPLES!)
ucl_2023_24 = "https://fbref.com/en/comps/8/2023-2024/schedule/2023-2024-Champions-League-Scores-and-Fixtures"
ucl_2024_25 = "https://fbref.com/en/comps/8/2024-2025/schedule/2024-2025-Champions-League-Scores-and-Fixtures"

# SCRAPE BOTH SEASONS
df_2023_24 = scrape_ucl(ucl_2023_24)
df_2024_25 = scrape_ucl(ucl_2024_25)

print("\n2023–24 Matches:", len(df_2023_24))
print(df_2023_24.head())

print("\n2024–25 Matches:", len(df_2024_25))
print(df_2024_25.head())

# Save CSVs
df_2023_24.to_csv("ucl_2023_24.csv", index=False)
df_2024_25.to_csv("ucl_2024_25.csv", index=False)


HTTPError: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/8/2023-2024/schedule/2023-2024-Champions-League-Scores-and-Fixtures